In [1]:
import torch
import tiktoken

In [2]:
with open( "humphreys.txt", "r" ) as f:
    raw_text = f.read()

raw_text[:50]

'ABOUT fifteen years ago, on a date late in August '

In [3]:
tokenizer = tiktoken.get_encoding("gpt2")

In [4]:
enc_text = tokenizer.encode(raw_text)

In [5]:
print(enc_text[:20])

[6242, 12425, 17280, 812, 2084, 11, 319, 257, 3128, 2739, 287, 2932, 393, 1903, 287, 2693, 11, 257, 4512, 9859]


In [6]:
print( tokenizer.decode( enc_text[:2]))

ABOUT


In [7]:
len( enc_text)

14643

In [8]:
for i in range(1,10):
    print("Input:", tokenizer.decode(enc_text[:i]), "Target:", tokenizer.decode([enc_text[i]]))

Input: AB Target: OUT
Input: ABOUT Target:  fifteen
Input: ABOUT fifteen Target:  years
Input: ABOUT fifteen years Target:  ago
Input: ABOUT fifteen years ago Target: ,
Input: ABOUT fifteen years ago, Target:  on
Input: ABOUT fifteen years ago, on Target:  a
Input: ABOUT fifteen years ago, on a Target:  date
Input: ABOUT fifteen years ago, on a date Target:  late


In [9]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [11]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[ 6242, 12425, 17280,   812],
        [ 2084,    11,   319,   257],
        [ 3128,  2739,   287,  2932],
        [  393,  1903,   287,  2693],
        [   11,   257,  4512,  9859],
        [  510,   379,  5187,   301],
        [17899,   431,    11,   257],
        [ 1499,  4429,   287,  8345]])

Targets:
 tensor([[12425, 17280,   812,  2084],
        [   11,   319,   257,  3128],
        [ 2739,   287,  2932,   393],
        [ 1903,   287,  2693,    11],
        [  257,  4512,  9859,   510],
        [  379,  5187,   301, 17899],
        [  431,    11,   257,  1499],
        [ 4429,   287,  8345,  4492]])
